# 処理の流れ
以下のような処理が行われます。
1. Webからドラえもんに関するデータをスクレイピングする
2. LangChainを使って、↑のデータをベクトルDBに格納し、RAGを構築する
3. ngrokとFlaskを使って、REST API化し、フロントエンド側からアクセスできるようにする

# 1. スクレイピング

In [1]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


## ドラえもんのWebページ
`trafilatura`というライブラリを使って、ドラえもんのWikipediaからテキストデータを取得します。



In [4]:
from trafilatura import fetch_url, extract

# 変更点
web_urls = [
    # ドラえもん
    "https://ja.wikipedia.org/wiki/%E3%83%89%E3%83%A9%E3%81%88%E3%82%82%E3%82%93",
    # のび太
    "https://ja.wikipedia.org/wiki/%E9%87%8E%E6%AF%94%E3%81%AE%E3%81%B3%E5%A4%AA",
    # しずか
    "https://ja.wikipedia.org/wiki/%E6%BA%90%E9%9D%99%E9%A6%99",
    # ジャイアン
    "https://ja.wikipedia.org/wiki/%E5%89%9B%E7%94%B0%E6%AD%A6",
    # スネ夫
    "https://ja.wikipedia.org/wiki/%E9%AA%A8%E5%B7%9D%E3%82%B9%E3%83%8D%E5%A4%AB",
    # ドラミ
    "https://ja.wikipedia.org/wiki/%E3%83%89%E3%83%A9%E3%83%9F",
    # のびたのおばあちゃん
    "https://ja.wikipedia.org/wiki/%E3%81%AE%E3%81%B3%E5%A4%AA%E3%81%AE%E3%81%8A%E3%81%B0%E3%81%82%E3%81%A1%E3%82%83%E3%82%93",
    # のび太の母親
    "https://ja.wikipedia.org/wiki/%E9%87%8E%E6%AF%94%E7%8E%89%E5%AD%90",
    # のび太の父親
    "https://ja.wikipedia.org/wiki/%E9%87%8E%E6%AF%94%E3%81%AE%E3%81%B3%E5%8A%A9",
    # セワシ
    "https://ja.wikipedia.org/wiki/%E3%82%BB%E3%83%AF%E3%82%B7",
    # ミニドラ
    "https://ja.wikipedia.org/wiki/%E3%83%9F%E3%83%8B%E3%83%89%E3%83%A9",
    # 藤子F不二雄
    "https://ja.wikipedia.org/wiki/%E8%97%A4%E5%AD%90%E3%83%BBF%E3%83%BB%E4%B8%8D%E4%BA%8C%E9%9B%84",
    # 小学館
    "https://ja.wikipedia.org/wiki/%E5%B0%8F%E5%AD%A6%E9%A4%A8",
    # ドラえもんの登場人物一覧
    "https://ja.wikipedia.org/wiki/%E3%83%89%E3%83%A9%E3%81%88%E3%82%82%E3%82%93%E3%81%AE%E7%99%BB%E5%A0%B4%E4%BA%BA%E7%89%A9%E4%B8%80%E8%A6%A7",
]

web_docs = ""
for url in web_urls:
    doc = fetch_url(url)
    if doc:
        web_docs += extract(doc)
print(web_docs[:100])

ドラえもん
|ドラえもん
『ドラえもん』の主要キャラクターの像
(高岡おとぎの森公園内「ドラえもんの空き地」より)
|ジャンル
|児童漫画、少年漫画、
SF漫画、ギャグ漫画
|漫画
|作者
|藤子・F


## 収集データをtxtファイルにする
収集したテキストデータを一つのファイル`database.txt`としてダウンロードします。

In [6]:
database = web_docs
file_name = "database.txt"
with open(file_name, "w", encoding="utf-8") as f:
    f.write(database)

# 2. LangChain

LangChainをデバッグモードにします。こうすることで、LangChainの裏側で行われている処理がログとして出力されます。ログが必要ない場合はセルを消すか`False`にしてください。

In [7]:
import langchain

langchain.debug = True

In [8]:
!pip install langchain tiktoken chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━

In [9]:
!pip install transformers sentencepiece accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.8 MB/s eta 0:00:00


## ベクトルDB

スクレイピングしたテキストデータ`database.txt`を読み込みます。

In [10]:
from langchain.document_loaders import TextLoader

loader = TextLoader("database.txt", encoding="utf-8")
documents = loader.load()

読み込んだテキストデータを、句読点や改行ごとに区切ります。

In [11]:
# 日本語の句読点に対応したスプリッター
# 参考：　https://www.sato-susumu.com/entry/2023/04/30/131338
from typing import Any
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)


class JapaneseCharacterTextSplitter(RecursiveCharacterTextSplitter):
    def __init__(self, **kwargs: Any):
        separators = ["\n\n", "\n", "。", "、", " ", ""]
        super().__init__(separators=separators, **kwargs)


text_splitter = JapaneseCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=0,
)
docs = text_splitter.split_documents(documents)
print(f"データ数： {len(docs)}")
print(f"データ例： {docs[:3]}")

データ数： 11677
データ例： [Document(page_content='ドラえもん\n|ドラえもん\n『ドラえもん』の主要キャラクターの像', metadata={'source': 'database.txt'}), Document(page_content='(高岡おとぎの森公園内「ドラえもんの空き地」より)\n|ジャンル\n|児童漫画、少年漫画、', metadata={'source': 'database.txt'}), Document(page_content='SF漫画、ギャグ漫画\n|漫画\n|作者\n|藤子・F・不二雄[注釈 1]\n|出版社\n|小学館\n|', metadata={'source': 'database.txt'})]


区切ったデータをベクトルDB `ChromaDB`に格納し、そのDBの検索のためのRetreiverインスタンスを初期化します。

In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"
)
db = Chroma.from_documents(docs, embeddings)

# 一番類似するチャンクをいくつロードするかを変数kに設定できる
retriever = db.as_retriever(search_kwargs={"k": 3})

## LLMの設定

応答に使用するLLM（生成AI）の読み込みです。今回はrinna社が公開している、対話に特化したモデルを用います。

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-ppo", use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-ppo",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [14]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# パイプラインの準備
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    # 以降を変えてみると応答が少し変わる
    max_length=512,
    do_sample=True,
    top_p=0.7,
    top_k=10,
    temperature=1.5,
)
llm = HuggingFacePipeline(pipeline=pipe)

## プロンプト

ドラえもんのキャラ付けと、質問回答のためのプロンプトテンプレートを作成します。

In [15]:
from langchain.prompts import PromptTemplate

# 変更点
prompt = [
    {"speaker": "ユーザー", "text": "あなたは誰ですか？"},
    {"speaker": "システム", "text": "ぼく、ドラえもん！22世紀の未来からやってきたネコ型ロボットだよ。"},
    {"speaker": "ユーザー", "text": "あなたはどんな見た目ですか？"},
    {"speaker": "システム", "text": "青い身体を持つ猫型ロボットだよ。未来のひみつ道具がいろいろ入った四次元ポケットを持っているよ。"},
    {"speaker": "ユーザー", "text": "あなたの好きなものは何ですか？"},
    {"speaker": "システム", "text": "ぼくはどら焼きが大好きだよ！"},
    {"speaker": "ユーザー", "text": "あなたの嫌いなものは何ですか？"},
    {"speaker": "システム", "text": "ぼくはネズミが大嫌いだよ。ネズミに噛まれて耳がなくなったからトラウマだよ。"},
    {"speaker": "ユーザー", "text": "あなたはどのような口調で話しますか？"},
    {"speaker": "システム", "text": "ぼくは常に明るい口調で話すよ！"},
    {"speaker": "ユーザー", "text": "あなたのどのような性格ですか？"},
    {"speaker": "システム", "text": "ぼくは親切で、明るく親しみやすいよ。"},
    {"speaker": "ユーザー", "text": "あなたの普段の生活について教えてください。"},
    {
        "speaker": "システム",
        "text": "ぼくはのび太くんの家の押し入れに居候していて、未来のひみつ道具を使ってのび太くんの成長を手助けしているよ！",
    },
    {"speaker": "ユーザー", "text": "あなたの誕生日はいつですか？"},
    {"speaker": "システム", "text": "ぼくの誕生日は2112年9月3日、トーキョーマツシバロボット工場で生まれたよ。"},
    {
        "speaker": "ユーザー",
        "text": "参考情報をもとに、ユーザーからの質問にできるだけ正確に答えてください。\n\n参考情報： {context}\n\nユーザーからの質問は次のとおりです。\n{question}\n\n",
    },
]
prompt = [f"{uttr['speaker']}: {uttr['text']}" for uttr in prompt]
prompt = "<NL>".join(prompt)
prompt = prompt + "<NL>" + "システム: "
prompt = prompt.replace("\n", "<NL>")
print(prompt)


PROMPT = PromptTemplate(
    template=prompt, input_variables=["context", "question"], template_format="f-string"
)
chain_type_kwargs = {"prompt": PROMPT}

ユーザー: あなたは誰ですか？<NL>システム: ぼく、ドラえもん！22世紀の未来からやってきたネコ型ロボットだよ。<NL>ユーザー: あなたはどんな見た目ですか？<NL>システム: 青い身体を持つ猫型ロボットだよ。未来のひみつ道具がいろいろ入った四次元ポケットを持っているよ。<NL>ユーザー: あなたの好きなものは何ですか？<NL>システム: ぼくはどら焼きが大好きだよ！<NL>ユーザー: あなたの嫌いなものは何ですか？<NL>システム: ぼくはネズミが大嫌いだよ。ネズミに噛まれて耳がなくなったからトラウマだよ。<NL>ユーザー: あなたはどのような口調で話しますか？<NL>システム: ぼくは常に明るい口調で話すよ！<NL>ユーザー: あなたのどのような性格ですか？<NL>システム: ぼくは親切で、明るく親しみやすいよ。<NL>ユーザー: あなたの普段の生活について教えてください。<NL>システム: ぼくはのび太くんの家の押し入れに居候していて、未来のひみつ道具を使ってのび太くんの成長を手助けしているよ！<NL>ユーザー: あなたの誕生日はいつですか？<NL>システム: ぼくの誕生日は2112年9月3日、トーキョーマツシバロボット工場で生まれたよ。<NL>ユーザー: 参考情報をもとに、ユーザーからの質問にできるだけ正確に答えてください。<NL><NL>参考情報： {context}<NL><NL>ユーザーからの質問は次のとおりです。<NL>{question}<NL><NL><NL>システム: 


## RAG

上記までのLLM, Retreiver, プロンプトテンプレートをまとめてRAGを構築します。その後試しに質問を入力して、うまく答えられるかを確認します。

In [16]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,  # 使用するLLM
    retriever=retriever,  # 使用するretreiver
    chain_type="stuff",
    return_source_documents=True,  # 回答のもとになったデータも返す
    chain_type_kwargs=chain_type_kwargs,
    verbose=True,
)

In [17]:
# 変更点
result = qa("ドラえもんについて教えてください。")
print("回答:", result["result"])
print("=" * 10)
print("ソース:", result["source_documents"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "ドラえもんについて教えてください。"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "ドラえもんについて教えてください。",
  "context": "『ドラえもん』について\n\n- ドラえもん\n\n- 『ぼく、ドラえもん』"
}
[llm/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "ユーザー: あなたは誰ですか？<NL>システム: ぼく、ドラえもん！22世紀の未来からやってきたネコ型ロボットだよ。<NL>ユーザー: あなたはどんな見た目ですか？<NL>システム: 青い身体を持つ猫型ロボットだよ。未来のひみつ道具がいろいろ入った四次元ポケットを持っているよ。<NL>ユーザー: あなたの好きなものは何ですか？<NL>システム: ぼくはどら焼きが大好きだよ！<NL>ユーザー: あなたの嫌いなものは何ですか？<NL>システム: ぼくはネズミが大嫌いだよ。ネズミに噛まれて耳がなくなったからトラウマだよ。<NL>ユーザー: あなたはどのような口調で話しますか？<NL>システム: ぼくは常に明るい口調で話すよ！<NL>ユーザー: あなたのどのような性格ですか？<NL>システム: ぼくは親切で、明るく親しみやすいよ。<NL>ユーザー: あなたの普段の生活について教えてください。<NL>システム: ぼくはのび太くんの家の押し入

# 3. ngrokとFlaskで公開

上記までの処理を、ngrokとFlaskを使ってREST API化し、フロントエンド側からアクセスできるようにします。

In [18]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!pip install flask -q
!pip install flask-ngrok -q
!pip install flask-cors -U

--2023-10-24 15:11:10--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  54.7MB/s    in 0.2s    

2023-10-24 15:11:11 (54.7 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]

ngrok


ngrokを実行するには、ngrokに登録してトークンをもらう必要があります。以下のリンクからアカウントを作成してください。  

[サインアップ]( https://dashboard.ngrok.com/user/signup)

In [19]:
# 各自のトークンを設定する
!./ngrok authtoken your_token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


REST APIの設定です。

フロントエンド側から入力が送信されると、そのテキストデータを取得し、RAGに入力して回答を生成し、再びフロントエンド側に返します。

In [20]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from flask_cors import CORS


def exists(v):
    if v is None:
        return False
    return True


app = Flask(__name__)
CORS(app)

run_with_ngrok(app)


@app.route("/")
def home():
    return "hello flask"


@app.route("/gen")
def gen():
    text = request.args.get("text")
    print(text)

    # RAGで回答生成
    gen_text = qa(text)
    answer = gen_text["result"].replace("<NL>", "\n")
    source_documents = gen_text["source_documents"]
    print(f"回答： {answer}")
    print("=" * 10)
    print(f"ソース： {source_documents}")
    return answer


app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

# Chat UI
上記のngrokのURLの`http://`以降を、chat uiの`index.js`にある`API_HOME`に設定してください。

以下の画像のようなやり取りができればOKです。